In [1]:
import urllib.parse
import requests
import json

# Configs

In [2]:
HOST = "https://retailer.li-lady.com"
ACCESS_TOKEN = "X9cwAtYGvMGcSPVhQH7015N9C_zAOyNjOr_bSQpOnl-"

# Functions

In [3]:
def getCompetitionList(host: str, is_robot: bool = True):
    if is_robot:
        # for robot only competitions
        api = "/api/competition/robotlist"
    else:
        # can also play a standard competition using the APIs
        api = "/api/competition/studentlist"       
    
    url = urllib.parse.urljoin(host, api)
    res = requests.post(url=url)
    if res.status_code == 200:
        print(res.status_code)
        return res.json().get('data')
    else:
        print(res.status_code, res.json())
        return None

In [4]:
def getCompetitionData(host: str, access_token: str, competition_id: str):
    api = "/api/clientCompetitionData"
    url = urllib.parse.urljoin(host, api)
    headers = {"Authorization": "Bearer " + access_token}
    payload = {"competition_id": competition_id}
    res = requests.post(url=url, json=payload, headers=headers)

    if res.status_code == 200:
        print(res.status_code)
        return res.json()
    else:
        print(res.status_code, res.json())
        return None

In [5]:
def getRoundData(host: str, access_token: str, competition_id: str, round: int | None = None):
    api = "/api/clientRoundData"
    url = urllib.parse.urljoin(host, api)
    headers = {"Authorization": "Bearer " + access_token}
    payload = {"competition_id": competition_id}
    if round is not None:
        payload["round"] = round
    res = requests.post(url=url, json=payload, headers=headers)

    if res.status_code == 200:
        print(res.status_code)
        return res.json()
    else:
        print(res.status_code, res.json())
        return None

In [6]:
def postPriceAction(host: str, access_token: str, competition_id: str, price_action: int):
    api = "/api/competitionRoundPriceAction"
    url = urllib.parse.urljoin(host, api)
    headers = {"Authorization": "Bearer " + access_token}
    payload = {"competition_id": competition_id, "priceAction": price_action}

    res = requests.post(url=url, json=payload, headers=headers)

    if res.status_code == 200:
        print(res.status_code)
        return res.json()
    else:
        print(res.status_code, res.json())
        return None

In [7]:
# get the list of competitions for robots
competition_list = getCompetitionList(host=HOST, is_robot=True)
print("Competitions for robots: \n", competition_list)

200
Competitions for robots: 
 [{'competition_id': 'TEST_ROBOT', 'owner': 'tong.wang@nus.edu.sg'}, {'competition_id': 'ROBOT', 'owner': 'tong.wang@nus.edu.sg'}, {'competition_id': 'ROBOT_LONG', 'owner': 'tong.wang@nus.edu.sg'}]


In [8]:
# get competition data (for a new competition)
competition_data = getCompetitionData(host=HOST, access_token=ACCESS_TOKEN, competition_id="TEST_ROBOT")
print("Competition_data: \n", competition_data)

200
Competition_data: 
 {'student': {'name': 'Tong (Test)', 'email': 'tongwang@nus.edu.sg'}, 'data': {'competition_id': 'TEST_ROBOT', 'totalRounds': 6, 'isRobot': True, 'isStarted': False}, 'status': 0}


In [9]:
# get round data (for a new competition)
round_data = getRoundData(host=HOST, access_token=ACCESS_TOKEN, competition_id="TEST_ROBOT")
print("round_data: \n", round_data)

200
round_data: 
 {'student': {'name': 'Tong (Test)', 'email': 'tongwang@nus.edu.sg'}, 'data': {'competition_id': 'TEST_ROBOT', 'totalRounds': 6, 'isRobot': True, 'isStarted': False}, 'status': 0}


In [10]:
# post the 1st price action (Week 1 price is fixed at 0, the input has no effect)
round_data = postPriceAction(host=HOST, access_token=ACCESS_TOKEN, competition_id="TEST_ROBOT", price_action=2)
print("Round data: \n", round_data)

200
Round data: 
 {'student': {'name': 'Tong (Test)', 'email': 'tongwang@nus.edu.sg'}, 'data': {'competition_id': 'TEST_ROBOT', 'totalRounds': 6, 'isRobot': True, 'currentRound': 1, 'displayedRound': 1, 'clientRoundData': {'weeks': [{'price': 60, 'sales': 56, 'revenue': 3360, 'inventory': 1944}], 'isCompleted': False, 'priceActions': [0]}, 'isCompleted': False, 'isStarted': True, 'isView': True}, 'status': 0}


In [11]:
# fix price_action at 2 and run all the way to the end
price_action = 2
while not round_data['data']['isCompleted']:
    round_data = postPriceAction(host=HOST, access_token=ACCESS_TOKEN, competition_id="TEST_ROBOT", price_action=price_action)
    print(round_data)
    # you need to decide your own price strategy here
    price_action = 2  

200
{'student': {'name': 'Tong (Test)', 'email': 'tongwang@nus.edu.sg'}, 'data': {'competition_id': 'TEST_ROBOT', 'totalRounds': 6, 'isRobot': True, 'currentRound': 1, 'displayedRound': 1, 'clientRoundData': {'weeks': [{'price': 60, 'sales': 56, 'revenue': 3360, 'inventory': 1944}, {'price': 48, 'sales': 115, 'revenue': 5520, 'inventory': 1829}], 'isCompleted': False, 'priceActions': [0, 2]}, 'isCompleted': False, 'isStarted': True, 'isView': True}, 'status': 0}
200
{'student': {'name': 'Tong (Test)', 'email': 'tongwang@nus.edu.sg'}, 'data': {'competition_id': 'TEST_ROBOT', 'totalRounds': 6, 'isRobot': True, 'currentRound': 1, 'displayedRound': 1, 'clientRoundData': {'weeks': [{'price': 60, 'sales': 56, 'revenue': 3360, 'inventory': 1944}, {'price': 48, 'sales': 115, 'revenue': 5520, 'inventory': 1829}, {'price': 48, 'sales': 101, 'revenue': 4848, 'inventory': 1728}], 'isCompleted': False, 'priceActions': [0, 2, 2]}, 'isCompleted': False, 'isStarted': True, 'isView': True}, 'status': 0

In [12]:
# get competition data (for a new competition)
competition_data = getCompetitionData(host=HOST, access_token=ACCESS_TOKEN, competition_id="TEST_ROBOT")
print("Competition_data: \n", competition_data)

200
Competition_data: 
 {'student': {'name': 'Tong (Test)', 'email': 'tongwang@nus.edu.sg'}, 'data': {'competition_id': 'TEST_ROBOT', 'totalRounds': 6, 'isRobot': True, 'currentRound': 6, 'clientRoundDataArr': [{'revenue': 91497, 'optimal': 91497}, {'revenue': 65259, 'optimal': 65259}, {'revenue': 73922, 'optimal': 73922}, {'revenue': 96648, 'optimal': 108864}, {'revenue': 92269, 'optimal': 92269}, {'revenue': 79010, 'optimal': 79010}], 'isStarted': True, 'isCompleted': True}, 'status': 0}
